# EDA

### Insights we are seeking to get out of the analysis is to understand the purchasing patterns of the segments, Which would Answer Questions such as

### Which Segment has more monetary value
### Which Segment Purchases more often
### Which Brands do each segment prefer more often
### Who are the most valuable customers(Overall and to each brand) and which segment do they belong to

In [51]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
%matplotlib inline

In [52]:
df_org = pd.read_excel("Purchase Data - Assignment Detail.xlsx")

In [53]:
df = df_org.copy()

In [54]:
df.tail()

,Unnamed: 0,ID,Day,Incidence,Brand,Quantity,Last_Inc_Brand,Last_Inc_Quantity,Price_1,Price_2,...,Promotion_3,Promotion_4,Promotion_5,Sex,Marital status,Age,Education,Income,Occupation,Settlement size
58688,58688,200000500,703,0,0,0,2,1,1.41,1.85,...,1,0,0,0,0,42,1,120946,1,0
58689,58689,200000500,710,0,0,0,0,0,1.36,1.84,...,0,0,0,0,0,42,1,120946,1,0
58690,58690,200000500,717,0,0,0,0,0,1.50,1.80,...,0,0,0,0,0,42,1,120946,1,0
58691,58691,200000500,722,1,2,3,0,0,1.51,1.82,...,0,0,0,0,0,42,1,120946,1,0
58692,58692,200000500,726,0,0,0,2,1,1.51,1.82,...,0,0,0,0,0,42,1,120946,1,0


In [55]:
df["spent"] = 0.0
for i in range(len(df)):
    if(df['Incidence'][i]==1):
        brand = int(df['Brand'][i])
        df["spent"][i] = int( df["Quantity"][i] )* ( df["Price_"+str(brand)][i] )
    else:
        pass

C:\Users\SANJEE~1\AppData\Local\Temp/ipykernel_39300/2618110722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["spent"][i] = int( df["Quantity"][i] )* ( df["Price_"+str(brand)][i] )


In [56]:
df_proc = pd.concat([df , pd.get_dummies(df["Brand"],  prefix = "Brand", prefix_sep = "_").drop("Brand_0" , axis=1)], axis=1)

In [57]:
df_proc.head()

,Unnamed: 0,ID,Day,Incidence,Brand,Quantity,Last_Inc_Brand,Last_Inc_Quantity,Price_1,Price_2,...,Education,Income,Occupation,Settlement size,spent,Brand_1,Brand_2,Brand_3,Brand_4,Brand_5
0,0,200000001,1,0,0,0,0,0,1.59,1.87,...,1,110866,1,0,0.0,0,0,0,0,0
1,1,200000001,11,0,0,0,0,0,1.51,1.89,...,1,110866,1,0,0.0,0,0,0,0,0
2,2,200000001,12,0,0,0,0,0,1.51,1.89,...,1,110866,1,0,0.0,0,0,0,0,0
3,3,200000001,16,0,0,0,0,0,1.52,1.89,...,1,110866,1,0,0.0,0,0,0,0,0
4,4,200000001,18,0,0,0,0,0,1.52,1.89,...,1,110866,1,0,0.0,0,0,0,0,0


In [58]:
df_proc.groupby(["ID"]).sum()["Brand_1"]

ID
200000001    1.0
200000002    0.0
200000003    0.0
200000004    0.0
200000005    0.0
            ... 
200000496    1.0
200000497    1.0
200000498    5.0
200000499    2.0
200000500    2.0
Name: Brand_1, Length: 500, dtype: float64

### Calculating summary of each individual which would help us during RFM analysis

In [59]:
df_id = pd.DataFrame(index=df["ID"].unique())

In [60]:
df_id["num_visits"]  =  df.groupby(["ID"]).count()['Unnamed: 0']
df_id["num_purchases"]   = df.groupby(["ID"]).sum()["Incidence"]
df_id["brand_one_purch"]  =  df_proc[df_proc["Brand"]==1].groupby("ID").sum()["spent"]
df_id["brand_two_purch"]  =   df_proc[df_proc["Brand"]==2].groupby("ID").sum()["spent"]
df_id["brand_three_purch"]  =  df_proc[df_proc["Brand"]==3].groupby("ID").sum()["spent"]
df_id["brand_four_purch"]   = df_proc[df_proc["Brand"]==4].groupby("ID").sum()["spent"]
df_id["brand_five_purch"]  =  df_proc[df_proc["Brand"]==5].groupby("ID").sum()["spent"]
df_id["toal_purchase_value"] = df_proc.groupby("ID").sum()["spent"]

In [61]:
df_id.head()

,num_visits,num_purchases,brand_one_purch,brand_two_purch,brand_three_purch,brand_four_purch,brand_five_purch,toal_purchase_value
200000001,101,9,2.94,17.75,NaN,7.86,5.41,33.96
200000002,87,11,NaN,4.71,NaN,37.03,NaN,41.74
200000003,97,10,NaN,27.34,NaN,NaN,4.93,32.27
200000004,85,11,NaN,1.57,NaN,NaN,31.39,32.96
200000005,111,13,NaN,NaN,NaN,NaN,54.97,54.97


### Calculating summary of segment wise

In [65]:
df_seg = pd.DataFrame(index=df_proc["segment"].unique())

KeyError: 'segment'

In [66]:
df_proc.columns


Index(['Unnamed: 0', 'ID', 'Day', 'Incidence', 'Brand', 'Quantity',
       'Last_Inc_Brand', 'Last_Inc_Quantity', 'Price_1', 'Price_2', 'Price_3',
       'Price_4', 'Price_5', 'Promotion_1', 'Promotion_2', 'Promotion_3',
       'Promotion_4', 'Promotion_5', 'Sex', 'Marital status', 'Age',
       'Education', 'Income', 'Occupation', 'Settlement size', 'spent',
       'Brand_1', 'Brand_2', 'Brand_3', 'Brand_4', 'Brand_5'],
      dtype='object')

In [67]:
df_proc["promotion_status"] = (df_proc["Promotion_1"] |df_proc["Promotion_2"]|df_proc["Promotion_3"]|df_proc["Promotion_4"]|df_proc["Promotion_5"])

In [68]:
df_seg["num_people"] =  df_proc.groupby(['segment', 'ID']).count().groupby(["segment"]).count()["Unnamed: 0"]
df_seg["num_visits"]  =  df_proc.groupby(["segment"]).count()['Unnamed: 0']
df_seg["num_purchases"]   = df_proc.groupby(["segment"]).sum()["Incidence"]
df_seg["brand_one_purch"]  =  df_proc[df_proc["Brand"]==1].groupby("segment").sum()["spent"]
df_seg["brand_two_purch"]  =  df_proc[df_proc["Brand"]==2].groupby("segment").sum()["spent"]
df_seg["brand_three_purch"]  =  df_proc[df_proc["Brand"]==3].groupby("segment").sum()["spent"]
df_seg["brand_four_purch"]   = df_proc[df_proc["Brand"]==4].groupby("segment").sum()["spent"]
df_seg["brand_five_purch"]  =  df_proc[df_proc["Brand"]==5].groupby("segment").sum()["spent"]
df_seg["total_purchase_value"] = df_proc.groupby('segment').sum()["spent"]

df_seg["purch_per_visit"] = df_seg["total_purchase_value"]/df_seg["num_visits"]

df_seg["purch_per_person"] = df_seg["total_purchase_value"]/df_seg["num_people"] 
df_seg["visit_per_person"] = df_seg["num_visits"] /df_seg["num_people"]
df_seg["brand_one_purch_per_person"] =  df_seg["brand_one_purch"] /df_seg["num_people"]
df_seg["brand_two_purch_per_person"] =  df_seg["brand_two_purch"] /df_seg["num_people"]
df_seg["brand_three_purch_per_person"] =  df_seg["brand_three_purch"] /df_seg["num_people"]
df_seg["brand_four_purch_per_person"] =  df_seg["brand_four_purch"] /df_seg["num_people"]
df_seg["brand_five_purch_per_person"] =  df_seg["brand_five_purch"] /df_seg["num_people"]
  
df_seg["promotion_1"] = df_proc[df_proc["promotion_status"]==1].groupby('segment').sum()["spent"]/df_seg["num_people"] 
df_seg["promotion_0"] = df_proc[df_proc["promotion_status"]==0].groupby('segment').sum()["spent"] /df_seg["num_people"] 

KeyError: 'segment'